In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [61.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,974 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu ja

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/2/athlete_events.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("athlete_events.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()


+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summ

In [4]:
#Create a temporary view.
df.createOrReplaceTempView('games')


In [5]:
# Get the number of distinct sports played in 2016
spark.sql("SELECT count(distinct sport) FROM games WHERE year=2016").show()


+---------------------+
|count(DISTINCT sport)|
+---------------------+
|                   34|
+---------------------+



In [6]:
# Get the number of distinct sports played in 2012
spark.sql("SELECT count(distinct sport) FROM games WHERE year=2012").show()


+---------------------+
|count(DISTINCT sport)|
+---------------------+
|                   32|
+---------------------+



In [7]:
# Write a subquery to get the distinct sports played in 2016 but not in 2012.
spark.sql("""
    SELECT DISTINCT sport
    FROM games
    WHERE year=2016 AND sport NOT IN
      (SELECT DISTINCT sport
      FROM games
      WHERE year=2012)
    """).show()


+------------+
|       sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



In [8]:
# Another option could be using an EXCEPT
spark.sql("""
    SELECT DISTINCT sport
    FROM games
    WHERE year=2016
    EXCEPT
        SELECT
      distinct
      sport
    FROM games
    WHERE year=2012
    """).show()


+------------+
|       sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



In [9]:
# Get the year of the earliest Olympics in our data
spark.sql("""SELECT min(year) FROM games""").show()


+---------+
|min(year)|
+---------+
|     1896|
+---------+



In [10]:
# Get the sports that were played in the earliest Olympic games in our data
spark.sql("""
      SELECT
      DISTINCT sport
      FROM games
      WHERE year=1896""").show()


+-------------+
|        sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



In [11]:
# Determine which sports were in both the 1896 Olympics and the 2016 Olympics
spark.sql("""
  SELECT
    DISTINCT
      a.sport
  FROM games a
    JOIN
        (SELECT DISTINCT sport FROM games WHERE year=1896) b
    ON
      a.sport=b.sport
  WHERE a.year=2016""").show()


+-------------+
|        sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



In [12]:
# Who are the top 5 Gold medal winners in the Olympics
spark.sql("""
SELECT
  Name,
  medal,
  count(*) as medalCount
FROM games
WHERE medal ='Gold' GROUP BY Name, medal
ORDER BY medalCount desc
LIMIT 5
""").show(truncate=False)


+----------------------------------+-----+----------+
|Name                              |medal|medalCount|
+----------------------------------+-----+----------+
|Michael Fred Phelps, II           |Gold |23        |
|Raymond Clarence "Ray" Ewry       |Gold |10        |
|Frederick Carlton "Carl" Lewis    |Gold |9         |
|Larysa Semenivna Latynina (Diriy-)|Gold |9         |
|Mark Andrew Spitz                 |Gold |9         |
+----------------------------------+-----+----------+



In [13]:
# Who are the top 5 Silver medal winners
spark.sql("""
SELECT
  Name,
  medal,
  count(*) as medalCount
FROM games
WHERE medal ='Silver' GROUP BY Name, medal
ORDER BY medalCount desc
LIMIT 5
""").show(truncate=False)


+-------------------------------+------+----------+
|Name                           |medal |medalCount|
+-------------------------------+------+----------+
|Shirley Frances Babashoff      |Silver|6         |
|Aleksandr Nikolayevich Dityatin|Silver|6         |
|Aleksandr Vladimirovich Popov  |Silver|6         |
|Mikhail Yakovlevich Voronin    |Silver|6         |
|Yang Yang                      |Silver|6         |
+-------------------------------+------+----------+



In [14]:
# Who are the top 5 Bronze medal winners
spark.sql("""
SELECT
  Name,
  medal,
  count(*) as medalCount
FROM games
WHERE medal ='Bronze' GROUP BY Name, medal
ORDER BY medalCount desc
LIMIT 5
""").show(truncate=False)


+------------------------+------+----------+
|Name                    |medal |medalCount|
+------------------------+------+----------+
|Harri Tapani Kirvesniemi|Bronze|6         |
|Heikki Ilmari Savolainen|Bronze|6         |
|Merlene Joyce Ottey-Page|Bronze|6         |
|Franziska van Almsick   |Bronze|6         |
|Aleksey Yuryevich Nemov |Bronze|6         |
+------------------------+------+----------+



In [15]:
import pandas as pd

pandas_df=spark.sql("""
SELECT
  Name,
  medal,
  count(*) as medalCount
FROM games
WHERE medal ='Gold' GROUP BY Name, medal
ORDER BY medalCount desc
LIMIT 5
""").toPandas()

pandas_df.to_csv('top5Gold.csv')
